# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-09-30 to 2021-09-30
You have 437 stocks
['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AJG', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLL', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'C', 'CAH', 'CARR', 'CAT', 'CB', 'CBRE', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DRE', 'DRI', 'DTE', 'DU

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-09-24 00:00:00+00:00,172.400,21.3900,215.66,146.92,107.070,122.30,124.37,339.23,622.71,178.53,...,82.88,63.32,57.59,60.01,135.62,125.520,151.23,559.480,60.25,204.73
2021-09-27 00:00:00+00:00,166.380,21.4700,218.46,145.37,107.720,122.63,120.58,334.51,603.65,176.91,...,85.38,62.25,59.30,60.73,134.30,125.570,151.54,548.580,62.89,197.80
2021-09-28 00:00:00+00:00,159.900,21.4300,214.78,141.91,107.340,121.29,118.74,324.21,578.77,171.29,...,84.31,61.42,59.92,59.14,128.92,122.540,150.85,531.180,62.74,192.77
2021-09-29 00:00:00+00:00,160.000,21.0100,216.77,142.83,108.840,122.50,119.46,323.53,577.70,168.84,...,83.63,62.67,59.88,58.43,127.78,123.040,149.36,525.670,63.74,194.99
2021-09-30 00:00:00+00:00,161.725,20.5447,213.07,142.92,110.025,122.41,120.49,323.53,582.69,170.31,...,84.02,62.75,59.34,58.67,126.95,123.135,148.82,523.635,63.36,198.12


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAL'].tail()

,,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
date,ticker,,,
2021-09-24 00:00:00+00:00,AAL,1.601259,-1.664676,1.173200
2021-09-27 00:00:00+00:00,AAL,1.632968,-0.959170,1.300032
2021-09-28 00:00:00+00:00,AAL,1.656749,-1.395157,1.387230
2021-09-29 00:00:00+00:00,AAL,1.680530,-0.776849,1.403084
2021-09-30 00:00:00+00:00,AAL,1.672603,0.214030,1.411011


In [6]:
all_factors.describe()

,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
count,5.331400e+05,5.331400e+05,5.331400e+05
mean,-2.265676e-19,1.432707e-19,2.032445e-19
std,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728092e+00,-1.728120e+00,-1.728092e+00
25%,-8.640460e-01,-8.640485e-01,-8.640485e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.640460e-01,8.640485e-01,8.640485e-01
max,1.728092e+00,1.728120e+00,1.728102e+00


In [7]:
all_factors.head()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2016-11-25 00:00:00+00:00 A                                 1.093930   
                          AAL                               1.331741   
                          AAP                               1.490281   
                          AAPL                             -1.395157   
                          ABBV                              0.832338   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2016-11-25 00:00:00+00:00 A                          1.514071   
                          AAL                       -0.586603   
                          AAP                       -0.879905   
                          AAPL                      -0.245739   
                          ABBV                       1.442727   

                                  annualzed_volatility_20_day  
date                      ticker                               
2016-11-25 00:00:00+00:00 A                          0.079271  
                          AAL                       -0.198177  
                          AAP                        1.617123  
                          AAPL                      -0.808562  
                          ABBV                       0.760999

In [8]:
all_factors.tail()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-09-30 00:00:00+00:00 YUM                              -0.031708   
                          ZBH                               0.420132   
                          ZBRA                             -1.252470   
                          ZION                              1.680530   
                          ZTS                              -0.404278   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2021-09-30 00:00:00+00:00 YUM                        0.142686   
                          ZBH                       -0.126832   
                          ZBRA                       1.395157   
                          ZION                      -1.656749   
                          ZTS                        0.927462   

                                  annualzed_volatility_20_day  
date                      ticker                               
2021-09-30 00:00:00+00:00 YUM                       -1.442719  
                          ZBH                       -1.545770  
                          ZBRA                       1.244543  
                          ZION                      -0.515257  
                          ZTS                        0.110978